## 今度こそFine-TuningしたVGG16で遊ぼう
** 転移学習**とはImageNetのような巨大なデータセットを使って学習した重みを別のニューラルネットワークにコピーして、再学習(**Fine-Tuning**)を行うことである。手元にあるデータセットが少ない場合、特に有効な手法である。今回は、17種類の花の画像分類するツールを作成したいが、データ数が多くないため、VGGと同じ構成のネットワークを用意し、学習済みの重みを初期値として新しいデータセットを対象にfine-tuningするよ。
<p>
参 考
* [転移学習について]https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html
* [前半のfine-tuningについて]https://github.com/aidiary/keras-examples/blob/master/vgg16/17flowers/finetuning.py
* [後半について]https://github.com/aidiary/keras-examples/blob/master/vgg16/17flowers/predict.py

In [2]:
import os
import shutil
import random
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [19]:
#ディレクトリの指定、なければ作成
IN_DIR = 'jpg'
TRAIN_DIR = 'train_images'
TEST_DIR = 'test_images'

if not os.path.exists(TRAIN_DIR):
    os.mkdir(TRAIN_DIR)

if not os.path.exists(TEST_DIR):
    os.mkdir(TEST_DIR)

In [20]:
#labels.txtのファイルを読み込んで、花の名前を入れるとその花の存在する範囲を返す関数を作るよ
# name => (start idx, end idx)
flower_dics = {}

with open('labels.txt') as fp:
    for line in fp:
        line = line.rstrip() #文字列を右から削除
        cols = line.split() #文字列を分割

        assert len(cols) == 3 #超お手軽にテストする。条件式がFalseならアラート

        start = int(cols[0])
        end = int(cols[1])
        name = cols[2]

        flower_dics[name] = (start, end)

In [21]:
# 花ごとのディレクトリを作成
for name in flower_dics:
    os.mkdir(os.path.join(TRAIN_DIR, name))
    os.mkdir(os.path.join(TEST_DIR, name))

In [22]:
# jpgをスキャン
for f in sorted(os.listdir(IN_DIR)):
    # image_0001.jpg => 1
    prefix = f.replace('.jpg', '') #.jpg を (空白) に置き換える
    idx = int(prefix.split('_',1)[1]) #'_'の前と後に分ける。その右の方を選び、それが数字だからint型にする。

    for name in flower_dics:
        start, end = flower_dics[name]
        if idx in range(start, end + 1):
            source = os.path.join(IN_DIR, f) 
            dest = os.path.join(TRAIN_DIR, name)
            shutil.copy(source, dest)
            continue

In [23]:
# 訓練データの各ディレクトリからランダムに10枚をテストとする
for d in os.listdir(TRAIN_DIR):
    files = os.listdir(os.path.join(TRAIN_DIR, d)) #pathをtrain_images/[花]として１つ１つ繋げる
    random.shuffle(files) #花のファイル内の画像をシャッフル
    for f in files[:10]: #シャッフルしたファイルから10個選び、それをfに１つ１つ入れていく。
        source = os.path.join(TRAIN_DIR, d, f)  #train_image/[花]/[画像]のpathをつなぐ
        dest = os.path.join(TEST_DIR, d) #test_image/[花]をつなぐ
        shutil.move(source, dest) #sourceをdestファイルに移動

In [24]:
#結果を格納するディレクトリ作成
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

    
#結果を保存する関数の定義
def save_history(history, result_file):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    epochs = len(acc)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(epochs):
            fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))



In [25]:
classes = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus',
           'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower',
           'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup',
           'Windflower', 'Pansy']
batch_size = 32
nb_classes = len(classes)
img_rows, img_cols = 150, 150
channels = 3
epochs = 50
train_samples = 1190
val_samples = 170

In [26]:
# VGG16モデルと学習済み重みをロード
# 全結合層（FC）はいらないのでinclude_top=False
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層を構築
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

# VGG16とFCを接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

In [27]:
# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False

# Fine-tuningのときはSGDの方がよい？
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 1120 images belonging to 17 classes.
Found 160 images belonging to 17 classes.


In [29]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs = epochs)

Epoch 1/50
35/35 [==============================] - 135s 4s/step - loss: 3.0267 - acc: 0.0723 - val_loss: 2.6238 - val_acc: 0.1750
Epoch 2/50
35/35 [==============================] - 130s 4s/step - loss: 2.6152 - acc: 0.1652 - val_loss: 2.3623 - val_acc: 0.3187
Epoch 3/50
35/35 [==============================] - 131s 4s/step - loss: 2.3438 - acc: 0.2598 - val_loss: 2.0880 - val_acc: 0.4000
Epoch 4/50
35/35 [==============================] - 132s 4s/step - loss: 2.1037 - acc: 0.3634 - val_loss: 1.8052 - val_acc: 0.5437
Epoch 5/50
35/35 [==============================] - 131s 4s/step - loss: 1.8611 - acc: 0.4366 - val_loss: 1.5578 - val_acc: 0.5938
Epoch 6/50
35/35 [==============================] - 131s 4s/step - loss: 1.6617 - acc: 0.4893 - val_loss: 1.3477 - val_acc: 0.6500
Epoch 7/50
35/35 [==============================] - 131s 4s/step - loss: 1.4831 - acc: 0.5527 - val_loss: 1.1804 - val_acc: 0.6813
Epoch 8/50
35/35 [==============================] - 131s 4s/step - loss: 1.2992 - a

In [30]:
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))
save_history(history, os.path.join(result_dir, 'history_finetuning.txt'))

In [31]:
model.load_weights(os.path.join(result_dir, 'finetuning.h5'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input,decode_predictions

img_path = 'HIMAWARI.jpg'
img = image.load_img(img_path, target_size=(img_rows, img_cols))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

In [39]:
# クラスを予測
# 入力は1枚の画像なので[0]のみ
pred = model.predict(x)[0]

In [40]:
# 予測確率が高いトップ5を出力
top = 5
top_indices = pred.argsort()[-top:][::-1]
result = [(classes[i], pred[i]) for i in top_indices]
for x in result:
    print(x)

('Sunflower', 0.99974376)
('Daisy', 0.00024026785)
('Tulip', 8.3102677e-06)
('Pansy', 3.9450715e-06)
('ColtsFoot', 2.0047867e-06)
